## Визуальный анализ данных

### Описание датасета

__Статистические данные о ряде домов в Калифорнии, основанные на переписи 1990 года.__

longitude - долгота

latitude - широта

housing_median_age - средний возраст дома

total_rooms - общее количество комнат

total_bedrooms - общее количество спален

population - количество проживающих

households - домохозяйства

ocean_proximity - близость океана

median_income - средний доход

median_house_value - средняя стоимость дома

In [ ]:
df = pd.read_csv('housing.csv', sep=';')
df.head()
longitude	latitude	housing_median_age	total_rooms	total_bedrooms	population	median_income	median_house_value	ocean_proximity	INLAND
0	-122.23	37.88	41.0	880.0	129.0	322.0	8.3252	452600.0	NEAR BAY	0
1	-122.22	37.86	21.0	7099.0	1106.0	2401.0	8.3014	358500.0	NEAR BAY	0
2	-122.24	37.85	52.0	1467.0	190.0	496.0	7.2574	352100.0	NEAR BAY	0
3	-122.25	37.85	52.0	1274.0	235.0	558.0	5.6431	341300.0	NEAR BAY	0
4	-122.25	37.85	52.0	1627.0	280.0	565.0	3.8462	342200.0	NEAR BAY	0

In [ ]:
Распределение вещественных признаков
# {'red', 'green', 'blue'} - дискретный признак ('red', 'green', 'blue', 'red', 'green', 'red')
# [0, 100] - вещественный признак (0, 5, 5.6, 10.5, 10.57)
plt.figure(figsize=(8, 6))
plt.hist(df['median_house_value'])
plt.title('Распределение median house value')
plt.xlabel('Стоимость')
plt.ylabel('Количество');

In [ ]:
df_num_features = df.select_dtypes(include=['float64', 'float32', 'float16'])
df_num_features.drop('median_house_value', axis=1, inplace=True)

In [ ]:
df_num_features.hist(figsize=(16, 16), bins=20, grid=False);

In [ ]:
Поиск выбросов с помощью box plot

Как строится box plot



box - от 25% до 75% квантиля
линия в середине box - медиана
"усы"
Как строятся "усы" - вариантов масса

среднее +/- 3 сигма (стандартное отклонение)
min / max
median +/- 1.5*(q75 - q25),
...
*Интерквартильный размах = q75 - q25

plt.figure(figsize=(6, 3))

sns.boxplot(x=df['population'], whis=1.5)

plt.xlabel('population')
plt.show()

Анализ категориальных признаков

In [ ]:
counts = df['ocean_proximity'].value_counts()
    
plt.figure(figsize=(8, 6))    
plt.title('ocean_proximity')
sns.barplot(x=counts.index, y=counts.values)
    
plt.show()

In [ ]:
counts = df['INLAND'].value_counts()
    
plt.figure(figsize=(8, 6))    
plt.title('INLAND')
sns.barplot(x=counts.index, y=counts.values)
    
plt.show()

Анализ взаимных распределений


In [ ]:
sns.jointplot(x=df['median_income'], y=df['median_house_value'], kind='reg');

In [ ]:
sns.jointplot(x=df['latitude'], y=df['longitude'], kind='kde');

In [ ]:
sns.jointplot(x=df['latitude'], y=df['median_house_value'], kind='reg');

In [ ]:
Видно два города, имеет смысл посмотреть на зависимость по отдельности

df_cut = df[df['latitude'] < 36]
sns.jointplot(x=df_cut['latitude'], y=df_cut['median_house_value'], kind='reg');

Категориальные / бинарные признаки
box plot

In [ ]:
plt.figure(figsize=(16, 8))

sns.boxplot(x=df['median_house_value'], y=df['ocean_proximity'], whis=1.5)

plt.xlabel('median_house_value')
plt.ylabel('ocean_proximity')
plt.title('Distribution of median_house_value by ocean_proximity');

Как строится box plot

Подробное объяснение

box - от 25% до 75% квантиля

линия в середине box - медиана

"усы"
Как строятся "усы" - вариантов масса

среднее +/- 3 сигма (стандартное отклонение)

min / max

median +/- 1.5*(q75 - q25),
...

*Интерквартильный размах = q75 - q25

Матрица корреляций

In [ ]:
Показывает линейную связь между переменными
Изменяется от -1 до 1
Корреляция - мера только линейной связи
sns.jointplot(x=df['total_rooms'], y=df['total_bedrooms'], kind='reg');


In [ ]:
sns.jointplot(x=df['longitude'], y=df['latitude'], kind='reg');

In [ ]:
corr_matrix = df.corr()
corr_matrix = np.round(corr_matrix, 1)
corr_matrix[np.abs(corr_matrix) < 0.3] = 0
corr_matrix

In [ ]:
plt.figure(figsize=(10, 8))

sns.set(font_scale=1.4)

sns.heatmap(corr_matrix, annot=True, linewidths=.5, cmap='coolwarm')

plt.title('Correlation matrix');

Гео данные

In [ ]:
min_long = -124.55
max_long = -113.80

min_lat = 32.45
max_lat = 42.05

In [ ]:
import matplotlib.image as img
california_map = img.imread('California_Map.png')

plt.figure(figsize=(12, 9))
plt.imshow(california_map,
           extent=[min_long, max_long, min_lat, max_lat], alpha=0.5);

In [ ]:
plt.figure(figsize=(12, 9))

sc = plt.scatter(df['longitude'], df['latitude'], alpha=0.5, c=df['median_house_value'])

plt.imshow(california_map,
           extent=[min_long, max_long, min_lat, max_lat], alpha=0.5)

plt.colorbar(sc)
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.title("Homes geography");

Folium

In [ ]:
# !pip install folium
import folium
this_map = folium.Map(prefer_canvas=True)

def plotDot(point):
    folium.CircleMarker(
        location=[point.latitude, point.longitude],
        radius=2,
        popup=point.median_house_value
    ).add_to(this_map)
    
df.apply(plotDot, axis=1)

this_map.fit_bounds(this_map.get_bounds())

this_map

Kepler**

In [ ]:
# !conda install -c conda-forge keplergl
# !pip install keplergl==0.1.1
from keplergl import KeplerGl 

map_ = KeplerGl(height=700)
map_.add_data(df, 'Data')
map_.save_to_html(file_name='./california.html')
User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
Map saved to ./california.html!

### Summary
## Матрица корреляции

_Знакомство с признаками_

Плотность распределения (sns.distplot/sns.kdeplot/plt.hist - Узнать распределение признака)

sns.violinplot - Узнать распределение признака

Столбчатая диаграмма (sns.barplot/plt.bar - Узнать распределение категориального признака)

Ящик с усами (sns.boxplot/plt.boxplot - Узнать диапазон значений)

sns.jointplot - Взаимное изменение признаков

plt.pie - Изобразить долю объектов от всего кол-ва

Поиск выбросов


sns.distplot/sns.kdeplot/plt.hist - Искать хвосты слева и справа
sns.boxplot - Всё, что выходит за пределы усов - выбросы